# Import
* Ver01:
    * Loewner Framework基本框架
    * R/C vs. RC(τ)绘制
* Ver02:
    * 增加了Bootstrapping，对充满非线性元件的神经界面来说很合适
* Ver02_1
    * 原来的Lowess的平滑超参frac设置有问题，重新跑了一遍
    * 有些数据只跑了50的bootstrap，修改为100的
* Ver03 - New
    * 修改了DRT的有效值判定，共轭对目前可以全部标记为噪声，得到的数据就会非常干净，且物理意义明确
    * 添加了从LFDRT计算得到的IPDF -> CDF -> PDF的变换。相当于直接得到了可验证的Ground Truth



In [105]:
import os
import re
import gc
import sys 
from datetime import datetime
from loguru import logger



import torch
import numpy as np
import math
from scipy.linalg import eig, svd, solve
from statsmodels.nonparametric.smoothers_lowess import lowess


import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter


%matplotlib qt
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"



# LFDRT Pipeline

## Definition

### Loewner-Framework + Monte-Carlo

In [106]:

def Loewner_Framework(f, Z, REALFLAG = True):
    '''==================================================
        Construct Loewner Pencel
        Parameter: 
            f:          real array of frequency values
            Z:          complex array of impedance values (H = Z)
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            L:          Loewner matrix
            Ls:         Shifted Loewner matrix
            H_left:     Impedance values for group left
            H_right:    Impedance values for group right
        ==================================================
    '''
    _n = len(f)
    s = 2j * np.pi * f

    # Ensuring the input have an even number of elements 
    # for constructing the model having real entries
    if REALFLAG:
        if _n % 2 != 0:
            _n = _n - 1

    # Left & Right Data for Loewner Framework
    s_left  = s[:_n:2]
    H_left  = Z[:_n:2]
    s_right = s[1:_n:2] 
    H_right = Z[1:_n:2]

    # Construct complex conjugate values for ensuring model having real entries
    if REALFLAG:
        s_left  = np.stack([s_left, s_left.conj()], axis=1).flatten()
        H_left  = np.stack([H_left, H_left.conj()], axis=1).flatten()
        s_right = np.stack([s_right, s_right.conj()], axis=1).flatten()
        H_right = np.stack([H_right, H_right.conj()], axis=1).flatten()

    # Constructing the Loewner Matrix & Shifted Loewner Matrix
    # L   = (H_left[:,None] - H_right[None,:]) / (s_left[:,None] - s_right[None,:])
    # Ls  = (s_left[:,None] * H_left[:,None] - s_right[None,:] * H_right[None,:]) / (s_left[:,None] - s_right[None,:])
    L   = (H_left[None,:] - H_right[:,None]) / (s_left[None,:] - s_right[:,None])
    Ls  = (s_left[None,:] * H_left[None,:] - s_right[:,None] * H_right[:,None]) / (s_left[None,:] - s_right[:,None])

    # Transforming the conplex L & Ls to obtain matrices with real entries
    if REALFLAG:
        _J_diag = np.eye(_n//2)
        _J  = (1/np.sqrt(2)) * np.array([[1, 1j], [1, -1j]])
        _J  = np.kron(_J_diag, _J)

        L       = (_J.conj().T @ L @ _J).real
        Ls      = (_J.conj().T @ Ls @ _J).real
        H_left  = ((_J.T @ H_left).T).real
        H_right = (_J.conj().T @ H_right).real

        
    return L, Ls, H_left, H_right

def state_space_model(L, Ls, H_left, H_right):
    '''==================================================
        Construct state space model from Loewner Pencel
        Parameter: 
            L:          Loewner matrix
            Ls:         Shifted Loewner matrix
            H_left:     Impedance values for group left
            H_right:    Impedance values for group right
        Returen:
            Ek, Ak, Bk, Ck:
                Ek x' = Ak x + Bk u
                   y  = Ck x + Dk u (Dk = 0)
        ==================================================
    '''
    # rank of the Loewner Pencel
    _rank = np.linalg.matrix_rank(np.concatenate((L, Ls), axis=0))
    Y_L, svd_L, X_L = svd(L, full_matrices=False, lapack_driver='gesvd')
    X_L = X_L.T
    
    # Reduced state space model interpolating the data
    Yk = Y_L[:, :_rank]
    Xk = X_L[:, :_rank]

    Ek = -Yk.T@L@Xk
    Ak = -Yk.T@Ls@Xk
    Bk = Yk.T@H_right
    Ck = H_left.T@Xk

    return Ek, Ak, Bk, Ck

def DRT_Transform(Ek, Ak, Bk, Ck, REALFLAG = True, real_th = 1e3):
    '''==================================================
        Transform state space model to DRT model
        Parameter: 
            Ek, Ak, Bk, Ck:
                Ek x' = Ak x + Bk u
                   y  = Ck x + Dk u (Dk = 0)
        Returen:
            tau_i   tau_i from RC pair in DRT
            R_i:    R_i from RC pair in DRT
            C_i:    C_i from RC pair in DRT
        ==================================================
    '''
    # Solve Av= λEv & wT A= λ wT E & Res = CvwB/wEv, wEv =  δ
    _pol, _U = eig(Ak, Ek)     # 
    wB = solve(_U, solve(Ek,Bk))
    Cv = Ck @ _U
    _res = Cv * wB

    # Calculate R_i & tau_i
    tau_i   = (-1/_pol) 
    R_i     = (-_res / _pol)
    # C_i     = (1/_res)

    if REALFLAG:
        real_ratio = np.where(np.abs(tau_i.imag) == 0, np.inf, np.abs(tau_i.real / (tau_i.imag+1e-20)))       
        real_mask = (real_ratio > real_th) & (tau_i.real>0) & (R_i.real>0)
        tau_i   = tau_i[real_mask].real
        R_i     = R_i[real_mask].real
        C_i     = tau_i/R_i

    _idx = np.argsort(tau_i)
    tau_i = tau_i[_idx]
    R_i   = R_i[_idx]
    C_i   = C_i[_idx]

    return tau_i, R_i, C_i

def DRT_singularity_analysis(f, Z, REALFLAG = True):
    '''==================================================
        DRT Singularity Analysis
        Parameter: 
            f:          real array of frequency values
            Z:          complex array of impedance values (H = Z)
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            R_i:        R_i from RC pair in DRT
            tau_i:      tau_i from RC pair in DRT
        ==================================================
    '''
    L, Ls, H_left, H_right = Loewner_Framework(f, Z, REALFLAG)
    Y, svd_L, X = svd(np.concatenate([L, Ls]), full_matrices=False)

    return svd_L

def DRT_Calculation(f, Z, REALFLAG = True):
    '''==================================================
        DRT Analysis
        Parameter: 
            f:          real array of frequency values
            Z:          complex array of impedance values (H = Z)
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            R_i:        R_i from RC pair in DRT
            tau_i:      tau_i from RC pair in DRT
            H:          reconstructed impedance values
            res_ReZ:    relative error of real part of impedance values
            res_ImZ:    relative error of imaginary part of impedance values
        ==================================================
    '''
    L, Ls, H_left, H_right = Loewner_Framework(f, Z, REALFLAG)
    Ek, Ak, Bk, Ck = state_space_model(L, Ls, H_left, H_right)
    tau_i, R_i, C_i = DRT_Transform(Ek, Ak, Bk, Ck)

    return tau_i, R_i, C_i



def DRT_Bootstrap(f, Z, REALFLAG = True, n_batch = 100, n_point = 100, idx_base=1000):
    '''==================================================
        DRT Analysis with Bootstrap for one channel
        Parameter: 
            f:          frequency values with full range (5000)
            Z:          impedance values (H = Z)
            n_batch:    number of bootstrap samples
            n_point:    number of points for each sample
            REALFLAG:   boolean flag to indicate if the model should have real entries
        Returen:
            DRTdata:    list of tuples (tau_i, R_i, C_i) for each sample
            EISdata:    list of tuples (f, Z.real, Z.imag) for each sample
        ==================================================
    '''
    DRTdata = []
    EISdata = []
    for _ in range(n_batch):

        
        indices = np.arange(idx_base, f.shape[0])
        bins = np.array_split(indices, n_point)
        sampled_indices = [np.random.choice(bin, 1)[0] for bin in bins]
        
        f_sampled = f[sampled_indices]
        Z_sampled = Z[sampled_indices]

        _tau_i, _R_i, _C_i = DRT_Calculation(f_sampled, Z_sampled, REALFLAG)
        DRTdata.append((np.array([_tau_i, _R_i, _C_i])))
        EISdata.append((np.array([f_sampled, Z_sampled.real, Z_sampled.imag])))

    return DRTdata, EISdata



### Equidistribution

In [107]:
from scipy.interpolate import UnivariateSpline, PchipInterpolator

def DRT_Equidistribution(tau_list, tau_quant=[0.025, 0.975], frac=0.1):
    '''==================================================
        Resample tau based on equidistribution
        Parameter: 
            tau_list: ordered tau list from Monte Carlo
                -- list[n_mc][n_tau] 
            tau_quant: quantile for clipping the tau values
                -- array[2] (default: [0.025, 0.975])
            frac: smoothing fraction for lowess
                -- float range[0,1] (default: 0.1)
        Returen:
            tau_re_log:  equidistributed tau_log 
                -- array[n_tau_equidist]
            
        ==================================================
    '''
    # constructing dt vs. t & m(t) = 1/dt(t)
    t_dt_list   = [[0.5*(np.log10(_tau[:-1])+np.log10(_tau[1:])), 
                     np.diff(np.log10(_tau[:]))] for _tau in tau_list]
    t_dt_list   = np.concatenate(t_dt_list, axis=1) 
    _t  = t_dt_list[0,:]
    _dt = t_dt_list[1,:]

    _t, idx = np.unique(_t, return_index=True)
    _dt = _dt[idx]  


    _t_clip = np.quantile(_t, tau_quant)
    _t_mask = (_t >= _t_clip[0]) & (_t <= _t_clip[1])
    _t      = _t[_t_mask]
    _dt     = _dt[_t_mask]


    # Smoothing the dt vs. t then calculating m(t) = 1/dt(t)
    # Note here we calculate m in log space
    t_dt_smooth = lowess(_dt, _t, frac=frac)
    t_dt_smooth = t_dt_smooth.T
    _t  = t_dt_smooth[0,:]
    _m  = 1/t_dt_smooth[1,:]
    

    # Spline interpolation for C2 continuity
    smooth_s    = 1.4826 * np.median(np.abs(_m - np.median(_m))) 
    spl         = UnivariateSpline(_t, _m, s=smooth_s, k=3)
    _m_C2       = spl(_t)

    # Equidistribution - Trapz
    S = np.zeros_like(_t)
    S[1:] = np.cumsum(0.5*(_m_C2[1:]+_m_C2[:-1])*(_t[1:]-_t[:-1]))
    # Equidistribution - uniformlly sampled inverse CDF
    n_tau_equidist = len(_t)
    xi = np.linspace(0.0, 1.0, n_tau_equidist)
    S_end = S[-1]
    S_target = xi * S_end
    # Equidistribution - resample tau
    tau_re_log = np.interp(S_target, S, _t)
    # tau_re = np.power(10.0, tau_re_log)

    return tau_re_log



def DRT_IPDF_Resample(ipdf_x, ipdf_p, x_equidist_log, frac=0.05):
    '''==================================================
        Resample the inverse probability distribution function (IPDF)
        Parameter: 
            ipdf_x:     x values of the IPDF
                -- array[n_ipdf]
            ipdf_p:     p values of the IPDF
                -- array[n_ipdf]
            x_equidist_log: target x values for resampling
                -- array[n_target]
        Returen:
            pdf_x:      x values of the PDF after resampling
                -- array[n_target]
            ipdf_est:   estimated p values of the PDF after resampling
                -- array[n_target]
        ==================================================
    '''
    idx = np.argsort(ipdf_x)

    ipdf_x_log = np.log10(ipdf_x)[idx]
    
    ipdf_p = np.clip(ipdf_p, 1e-16, None)[idx]
    # ipdf_p = np.array(ipdf_p)[idx]

    # 确保概率非负并归一化
    ipdf_sum = ipdf_p.sum()
    if ipdf_sum > 0:
        ipdf_p = ipdf_p / ipdf_sum
    # 计算累积分布函数 (CDF) 在每个边界处的值
    ipdf_p_log = np.log10(ipdf_p)
    
    loess_result = lowess(ipdf_p_log, ipdf_x_log, frac=frac)
    
    intp_mask = (x_equidist_log>=loess_result[:,0].min()) & (x_equidist_log<=loess_result[:,0].max())
    x_equidist_log = x_equidist_log[intp_mask]
    ipdf_est = np.interp(x_equidist_log, loess_result[:,0], loess_result[:,1])

    x_equidist = np.power(10, x_equidist_log)
    ipdf_est = np.power(10, ipdf_est)

    ipdf_est_sum = np.sum(ipdf_est)
    # ipdf_sum = ipdf_sum / ipdf_est_sum
    ipdf_sum = ipdf_sum 


    return x_equidist, ipdf_est * ipdf_sum



def DRT_IPDF2PDF(ipdf_x, ipdf_p, pdf_x):
    '''==================================================
        Resample the inverse probability distribution function (IPDF)
        Parameter: 
            ipdf_x:     x values of the IPDF
                -- array[n_ipdf]
            ipdf_p:     p values of the IPDF
                -- array[n_ipdf]
            pdf_x: target x values for PDF
                -- array[n_pdf]
        Returen:
            pdf_x:      x values of the PDF after resampling
                -- array[n_pdf]
            pdf_p:      estimated p values of the PDF after resampling
                -- array[n_pdf]
        ==================================================
    '''

    idx = np.argsort(ipdf_x)
    x_sorted = np.array(ipdf_x)[idx]
    p_sorted = np.clip(np.array(ipdf_p)[idx], 1e-16, None)

    x_sorted, idx = np.unique(x_sorted, return_index=True)
    p_sorted = p_sorted[idx]   # 如果不平均直接取第一个

    n = len(x_sorted)
    p_total = p_sorted.sum()
    if p_total == 0:
        raise ValueError("ipdf_p sum is zero; cannot form CDF.")
    p_sorted = p_sorted / p_total


    cdf_vals = np.concatenate(([1e-16], np.cumsum(p_sorted)))
    edges = np.empty(n+1)
    x_sorted = np.log10(x_sorted)
    edges[1:n] = (x_sorted[:-1] + x_sorted[1:]) / 2.0
    edges[0] = x_sorted[0] - (x_sorted[1] - x_sorted[0]) / 2 if n > 1 else x_sorted[0] - 0.5
    edges[n] = x_sorted[-1] + (x_sorted[-1] - x_sorted[-2]) / 2 if n > 1 else x_sorted[-1] + 0.5
    log_edges = edges
    
    # Fitting CDF
    cdf_vals_log = np.log10(cdf_vals)
    pchip = PchipInterpolator(log_edges, cdf_vals_log, extrapolate=True)
    pdf_x_log = np.log10(pdf_x)
    cdf_fitted = pchip(pdf_x_log)
    cdf_fitted[np.isnan(cdf_fitted)] = 1e-16 # extrapolate with 1e-16


    # dCDF/dlogx = PDF
    d_cdf_dlogx = pchip.derivative()(pdf_x_log)
    d_cdf_dlogx[np.isnan(d_cdf_dlogx)] = 1e-16 # extrapolate with 1e-16


    cdf_fitted = 10**cdf_fitted
    pdf_fitted = cdf_fitted * d_cdf_dlogx

    return pdf_x, pdf_fitted * p_total













### Helper

In [108]:


def DRT_Reconstruction_SSM(Ek, Ak, Bk, Ck, f, Z):
    '''==================================================
        Reconstruct DRT from state space model
        Parameter: 
            R_i:    R_i from RC pair in DRT
            tau_i   tau_i from RC pair in DRT
            f:  real array of frequency values
            Z:  complex array of impedance values (H = Z)
        Returen:
            H:  reconstructed impedance values
        ==================================================
    '''
    s = 2j * np.pi * f
    H = np.array([Ck @ solve(si * Ek - Ak, Bk) for si in s])
    res_ReZ = np.abs(((Z.real - H.real) / np.abs(Z))) * 100
    res_ImZ = np.abs(((Z.imag - H.imag) / np.abs(Z))) * 100

    return H, res_ReZ, res_ImZ


def DRT_Reconstruction_DRT(R_i, tau_i, f, Z):
    '''==================================================
        Reconstruct DRT from state space model
        Parameter: 
            Ek, Ak, Bk, Ck:
                Ek x' = Ak x + Bk u
                   y  = Ck x + Dk u (Dk = 0)
            f:  real array of frequency values
            Z:  complex array of impedance values (H = Z)
        Returen:
            H:  reconstructed impedance values
        ==================================================
    '''
    s = 2j * np.pi * f  # Broadcasting tau_i to match f
    _RC = R_i[None, :] / (1+s[:,None] * tau_i[None,:])
    H = np.sum(_RC, axis=1)

    res_ReZ = np.abs(((Z.real - H.real) / np.abs(Z))) * 100
    res_ImZ = np.abs(((Z.imag - H.imag) / np.abs(Z))) * 100

    return H, res_ReZ, res_ImZ




import numpy as np
from scipy.interpolate import RegularGridInterpolator

def _fill_nan_along_y(Z, y):
    """按列沿 y 方向线性填补 NaN；整列全 NaN 则置 0。"""
    Zf = Z.copy()
    ny, nx = Z.shape
    for j in range(nx):
        col = Zf[:, j]
        m = ~np.isnan(col)
        if m.any():
            Zf[:, j] = np.interp(y, y[m], col[m])
        else:
            Zf[:, j] = 0.0
    return Zf

def upsample_y_regular(X, Y, Z, ny_new=100, method='linear'):
    """
    仅沿 y 方向把 (ny, nx) -> (ny_new, nx)；x 保持原有（可为 logspace）。
    X, Y 可为 meshgrid(2D) 或各自 1D。
    返回 X_new, Y_new, Z_new（可直接 pcolormesh/imshow）。
    """
    # 规范 x,y 为 1D 网格坐标
    if X.ndim == 2 and Y.ndim == 2:
        x = X[0, :]
        y = Y[:, 0]
    else:
        x, y = np.asarray(X), np.asarray(Y)

    # 处理 NaN，避免插值器报错
    Zf = _fill_nan_along_y(np.asarray(Z, float), y)

    # 构建插值器（注意顺序是 (y, x)）
    rgi = RegularGridInterpolator((y, x), Zf, method=method, bounds_error=False, fill_value=np.nan)

    # 目标 y 网格（线性或你需要可改为对数）
    y_new = np.linspace(y.min(), y.max(), ny_new)
    X_new, Y_new = np.meshgrid(x, y_new)

    # 评估
    pts = np.stack([Y_new.ravel(), X_new.ravel()], axis=-1)
    Z_new = rgi(pts).reshape(Y_new.shape)
    return X_new, Y_new, Z_new




def DRT_Plot_Batch(fig, DRTdata_list, EISdata_list, IPDF_list, PDF_list, x_day, pdf_x, eis_seq, heat_range = [1,5.5]):
    
    axis = [0] * 6
    axis[0] = fig.add_subplot(2,3,1)    # Nyquist Plot
    axis[1] = fig.add_subplot(2,3,2)    # Bode Plot (Magnitude)
    axis[2] = fig.add_subplot(2,3,3)    # Bode Plot (Phase)
    axis[3] = fig.add_subplot(2,3,4)    # IPDF (Rτ)
    axis[4] = fig.add_subplot(2,3,5)    # PDF  (Rτ)
    axis[5] = fig.add_subplot(2,3,6)    # Heatmap  (Rτ)




    _s       = 2
    _alpha   = 0.7

    cmap = plt.colormaps.get_cmap('rainbow_r')
    for i in range(len(EISdata_list)):
        if i in eis_seq:
            ch_eis      = EISdata_list[i][0]
            ch_drt      = DRTdata_list[i]
            ch_ipdf     = IPDF_list[i]
            ch_pdf      = PDF_list[i]

            # ch_R    = np.array([i[1:,0] for i in ch_drt])
            # ch_C    = np.array([i[1:,-1] for i in ch_drt])
            ch_T    = np.concatenate([i[0,:] for i in ch_drt])
            ch_R    = np.concatenate([i[1,:] for i in ch_drt])
            ch_C    = np.concatenate([i[2,:] for i in ch_drt])

            _color  = cmap(i/len(EISdata_list))

            axis[0].plot(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2)
            axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2)
            axis[2].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2)

            # axis[3].scatter(ch_ipdf[1,:], ch_ipdf[0,:]/ch_ipdf[1,:]/ch_ipdf[1,:], s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')
            # axis[3].scatter(ch_ipdf[0,:], ch_ipdf[1,:], s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')
            axis[3].scatter(ch_R, ch_C, s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')
            
            # axis[4].scatter(ch_R, ch_C, s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')
            axis[4].scatter(ch_pdf[0,:], ch_pdf[1,:], s=_s, alpha=_alpha, color=_color, label=f'ch[{i:03d}]')

    # _idx
    heat_y   = x_day[eis_seq]
    heat_x   = PDF_list[eis_seq[0]][0,:]
    heat_PDF = np.array([PDF_list[i][1,:] for i in eis_seq])

    from scipy.interpolate import interp1d


    X, Y = np.meshgrid(heat_x, heat_y)
    Z = np.log10(heat_PDF)

    X, Y, Z = upsample_y_regular(X, Y, Z, ny_new=40, method='linear')

    Z_smooth = gaussian_filter(Z, sigma=1)
    axis[5].pcolormesh(X,Y,Z_smooth, shading='auto', cmap='rainbow', rasterized=True,
                       vmin = heat_range[0], vmax = heat_range[1])

    axis[5].set_xscale('log')



    # axis setting

    axis[0].set_aspect('equal', adjustable='datalim')
    xlim = axis[0].get_xlim()
    ylim = axis[0].get_ylim()

    # 取当前范围和 1e6 的最小值
    axis[0].set_xlim(xlim[0], min(xlim[1], 1e6))
    axis[0].set_ylim(ylim[0], min(ylim[1], 1e6))


    axis[1].set_ylim(1e3, 1e8)
    axis[1].grid(True, which='both', linestyle='--', linewidth=0.5)
    axis[2].set_ylim(-90, 0)
    axis[2].grid(True, which='both', linestyle='--', linewidth=0.5)

    
    axis[3].set_xscale('log')
    axis[3].set_yscale('log')
    # axis[3].set_ylim(1e1, 1e8)
    axis[3].grid(True, which='both', linestyle='--', linewidth=0.5)

    axis[4].set_xscale('log')
    axis[4].set_yscale('log')
    axis[4].set_ylim(1e1, 1e8)
    axis[4].grid(True, which='both', linestyle='--', linewidth=0.5)












In [109]:
# fig = plt.figure(figsize=(16, 9), constrained_layout=True)      
# DRT_Plot_Batch(fig, DRTdata_list, EISdata_list, IPDFdata_list, PDFdata_list, x_day_idx, pdf_x, eis_seq)

### Other

In [110]:

def DRT_Loess(DRTdata):
    '''==================================================
        DRT Analysis with Loess
        Parameter: 
            DRTdata:    list of tuples (tau_i, R_i, C_i) for each sample
        Returen:
            DRTdata_Loess:    Loess smoothed DRT data
        ==================================================
    '''

    # _tau_i  = np.concatenate([i[0,1:-1] for i in DRTdata])
    # _R_i    = np.concatenate([i[1,1:-1] for i in DRTdata])
    # _C_i    = np.concatenate([i[2,1:-1] for i in DRTdata])
    _tau_i  = np.concatenate([i[0,:] for i in DRTdata])
    _R_i    = np.concatenate([i[1,:] for i in DRTdata])
    _C_i    = np.concatenate([i[2,:] for i in DRTdata])

    # _order  = _tau_i.argsort()
    # _tau_i  = _tau_i[_order]
    # _R_i    = _R_i[_order]
    # _C_i    = _C_i[_order]

    x_log = np.log(_tau_i)
    y_log = np.log(_R_i)-np.log(_C_i)
    
    y_log_smooth = lowess(y_log, x_log, frac=0.1, it=3, return_sorted=False)


    RC_loess         = np.exp(y_log_smooth)
    tau_loess       = _tau_i
    DRTdata_Loess   =  np.array([tau_loess, RC_loess])

    return DRTdata_Loess



## Simulation Data

In [111]:
SIM_FLAG = False

### Data Simulation

In [112]:
if SIM_FLAG:

    # ECM_TYPE = 'R-(R||Q)'
    ECM_TYPE = 'C-R-(Q1 || (R-W))'
    # ECM_TYPE = 'R-(R||C)-(R||C)'
    # ECM_TYPE = 'C-R-W'


    if  ECM_TYPE == 'R-(R||Q)':    
        R0 = 100
        C0 = 1e-7

        
        R1 = 1000
        Y1 = 5e-8
        n1 = 0.9


        f = np.logspace(0,6,5000);  # Hz 

        Q0 = lambda x: 1/((1j*x*C0))
        Q1 = lambda x: 1/(Y1*(1j*x)**n1)


        noise_level = 1e-2
        Z_sim = np.array([ R0 + (1/(1/R1 + 1/Q1(w)))  for w in 2*np.pi*f])
        # Z_sim = np.array([ R0 + Q0(w) + (1/(1/R1 + 1/Q1(w)))  for w in 2*np.pi*f])
        Z_sim_noise = Z_sim + np.random.normal(0, noise_level, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, noise_level, Z_sim.shape) * Z_sim.imag

    elif ECM_TYPE == 'C-R-(Q1 || (R-W))':
        R0 = 10000
        C0 = 1e-7

        R1 = 1e5
        
        Y1 = 1e-10
        n1 = 0.9
        
        WR = 100
        WT = 1e-7
        n2 = 0.6

        f = np.logspace(0,6,5000);  # Hz 

        Q0 = lambda x: 1/((1j*x*C0))
        Q1 = lambda x: 1/(Y1*(1j*x)**n1)
        Q2 = lambda x: WR/((1j*x*WT)**n2)


        noise_level = 1e-2
        Z_sim = np.array([ Q0(w)+R0 + (1/(1/(R1+Q2(w)) + 1/(Q1(w)))) for w in 2*np.pi*f])
        Z_sim_noise = Z_sim + np.random.normal(0, noise_level, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, noise_level, Z_sim.shape) * Z_sim.imag

    elif ECM_TYPE == 'R-(R||C)-(R||C)':
        # Element
        R1 = 200; # Ω
        R2 = 100; # Ω
        R0 = 70;  # Ω

        C1 = 2.5e-3; # 
        C2 = 1e-4;   # 

        # tau1 = R1*C1;  # s
        # tau2 = R2*C2;  # s

        f = np.logspace(0,8,5000);  # Hz

        # Calculation of the impedance dataset 
        Z_sim = np.array([1/((1/R1)+1j*w*C1) +1/((1/R2)+1j*w*C2) +R0 for w in 2*np.pi*f])
        Z_sim_noise = Z_sim + np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, 0.001, Z_sim.shape) * Z_sim.imag



    elif ECM_TYPE == 'C-R-W':    
        R0 = 1000
        C0 = 1e-7

        
        WR = 100
        WT = 1e-1
        n1 = 0.5


        WR2 = 10000
        WT2 = 1e-3
        n2 = 0.1

        f = np.logspace(0,6,5000);  # Hz 

        Q0 = lambda x: 1/((1j*x*C0))
        Q1 = lambda x: WR/((1j*x*WT)**n1)
        Q2 = lambda x: WR/((1j*x*WT2)**n2)



        noise_level = 1e-2
        # Z_sim = np.array([ Q0(w)+R0 +Q1(w) +Q2(w)  for w in 2*np.pi*f])
        Z_sim = np.array([ R0 +Q1(w) +Q2(w)  for w in 2*np.pi*f])
        # Z_sim = np.array([ Q0(w)+R0 +Q1(w)  for w in 2*np.pi*f])
        # Z_sim = np.array([ Q0(w)+R0   for w in 2*np.pi*f])
        Z_sim_noise = Z_sim + np.random.normal(0, noise_level, Z_sim.shape) * Z_sim.real + 1j*np.random.normal(0, noise_level, Z_sim.shape) * Z_sim.imag





    plt.figure()
    plt.plot(Z_sim.real, -Z_sim.imag, label='Simulated Data')
    plt.plot(Z_sim_noise.real, -Z_sim_noise.imag, label='Simulated Data')
    # plt.loglog(f, np.abs(Z_sim), label='Ampletude')
    # plt.semilogx(f, np.angle(Z_sim)*180/np.pi, label='Ampletude')
    # plt.plot((1/Z_sim).real, (1/Z_sim).imag, label='Simulated Data')
    plt.title(ECM_TYPE)



### Run LF

In [113]:
if SIM_FLAG:
    
    
    REALFLAG = True
    n_batch = 100
    n_point = 100
    idx_base = 1000

    ## Noise
    _f = f
    _Z = Z_sim_noise
    DRTdata_n, EISdata_n = DRT_Bootstrap(_f, _Z, REALFLAG=REALFLAG, n_batch=n_batch, n_point=n_point, idx_base=idx_base)
    tau_list    = [_drt[0,0:-1] for _drt in DRTdata_n]
    R_list      = [_drt[1,0:-1] for _drt in DRTdata_n]
    # tau_list    = [_drt[0,:] for _drt in DRTdata_n]
    # R_list      = [_drt[1,:] for _drt in DRTdata_n]
    tau_re_log  = DRT_Equidistribution(tau_list,
                                       tau_quant=[0, 1], frac=0.1)

    R_list_all     = np.concatenate(R_list, axis=0) / len(R_list)
    tau_list_all   = np.concatenate(tau_list, axis=0)



    pdf_x = np.logspace(math.floor(np.log10(tau_list_all.min())), math.ceil(np.log10(tau_list_all.max())), 501)
    ipdf_x, ipdf_p  = DRT_IPDF_Resample(tau_list_all, R_list_all, tau_re_log, frac= 0.05)
    pdf_x, pdf_p    = DRT_IPDF2PDF(ipdf_x, ipdf_p, pdf_x)
    ipdf_p = ipdf_p * len(R_list)


    tau_n, R_n = tau_list_all, R_list_all * len(R_list)
    ipdf_x_n, ipdf_p_n = ipdf_x, ipdf_p
    pdf_x_n, pdf_p_n = pdf_x, pdf_p

    



    _f = f
    _Z = Z_sim
    DRTdata_o, EISdata_o = DRT_Bootstrap(_f, _Z, REALFLAG=REALFLAG, n_batch=n_batch, n_point=n_point, idx_base=idx_base)
    tau_list    = [_drt[0,1:-1] for _drt in DRTdata_o]
    R_list      = [_drt[1,1:-1] for _drt in DRTdata_o]
    # tau_list    = [_drt[0,:] for _drt in DRTdata_o]
    # R_list      = [_drt[1,:] for _drt in DRTdata_o]
    tau_re_log  = DRT_Equidistribution(tau_list,
                                       tau_quant=[0, 1], frac=0.1)

    R_list_all     = np.concatenate(R_list, axis=0) / len(R_list)
    tau_list_all   = np.concatenate(tau_list, axis=0)



    pdf_x = np.logspace(math.floor(np.log10(tau_list_all.min())), math.ceil(np.log10(tau_list_all.max())), 501)
    ipdf_x, ipdf_p  = DRT_IPDF_Resample(tau_list_all, R_list_all, tau_re_log, frac= 0.05)
    pdf_x, pdf_p    = DRT_IPDF2PDF(ipdf_x, ipdf_p, pdf_x)
    ipdf_p = ipdf_p * len(R_list)

    tau_o, R_o = tau_list_all, R_list_all * len(R_list)
    ipdf_x_o, ipdf_p_o = ipdf_x, ipdf_p
    pdf_x_o, pdf_p_o = pdf_x, pdf_p
    


### Plot

In [114]:
if SIM_FLAG:

    fig = plt.figure(figsize=(16, 9), constrained_layout=False)
    


    axis = [0] * 6
    axis[0] = fig.add_subplot(2,3,1)    # Nyquist Plot
    axis[1] = fig.add_subplot(2,3,2)    # Bode Plot (Magnitude)
    axis[2] = fig.add_subplot(2,3,3)    # Bode Plot (Phase)
    axis[3] = fig.add_subplot(2,3,4)    # Text
    axis[4] = fig.add_subplot(2,3,5)    # DRT (RC)
    axis[5] = fig.add_subplot(2,3,6)    # DRT (Rτ)

    text_axis = axis[3]
    text_axis.axis('off')


    axis[0].set_aspect('equal', adjustable='datalim')

    axis[1].set_ylim(1e3, 1e8)
    axis[1].grid(True, which='both', linestyle='--', linewidth=0.5)
    axis[2].set_ylim(-90, 0)
    axis[2].grid(True, which='both', linestyle='--', linewidth=0.5)

    
    axis[4].set_xscale('log')
    axis[4].set_yscale('log')
    axis[4].set_ylim(1, 1e8)
    axis[4].grid(True, which='both', linestyle='--', linewidth=0.5)



    axis[5].set_xscale('log')
    axis[5].set_yscale('log')







    _s       = 2
    _alpha   = 0.7

    cmap = plt.colormaps.get_cmap('rainbow_r')
    

    ch_eis      = EISdata_n[0]
    ch_drt      = DRTdata_n

    ch_T    = np.concatenate([i[0,:] for i in ch_drt])
    ch_R    = np.concatenate([i[1,:] for i in ch_drt])
    ch_C    = np.concatenate([i[2,:] for i in ch_drt])

    _color  = cmap(0)

    axis[0].plot(ch_eis[1,:], -ch_eis[2,:], color = _color, linewidth=2)
    axis[1].loglog(ch_eis[0,:], np.abs(ch_eis[1,:]+1j*ch_eis[2,:]), color = _color, linewidth=2)
    axis[2].semilogx(ch_eis[0,:], np.rad2deg(np.angle(ch_eis[1,:]+1j*ch_eis[2,:])), color = _color, linewidth=2)
    

    axis[4].scatter(pdf_x_o, pdf_p_o, s=_s, alpha=_alpha, color='blue', label=f'ch[{i:03d}]')
    # axis[4].scatter(pdf_x_n, pdf_p_n, s=_s, alpha=_alpha, color='red', label=f'ch[{i:03d}]')



    axis[5].scatter(tau_o, R_o, s=_s, alpha=_alpha, color='green', label=f'ch[{i:03d}]')
    axis[5].scatter(tau_n, R_n, s=_s, alpha=_alpha, color='orange', label=f'ch[{i:03d}]')

    axis[5].scatter(ipdf_x_o, ipdf_p_o, s=_s, alpha=_alpha, color='blue', label=f'ch[{i:03d}]')
    # axis[5].scatter(ipdf_x_o, ipdf_p_o, s=_s, alpha=_alpha, color='blue', label=f'ch[{i:03d}]')
    axis[5].scatter(ipdf_x_n, ipdf_p_n, s=_s, alpha=_alpha, color='red', label=f'ch[{i:03d}]')




# Data Loader

## Definition

In [115]:
def SearchELE(rootPath, ele_pattern = re.compile(r"(.+?)_归档")):
    '''==================================================
        Search all electrode directories in the rootPath
        Parameter: 
            rootPath: current search path
            ele_pattern: electrode dir name patten
        Returen:
            ele_list: list of electrode directories
        ==================================================
    '''
    ele_list = []
    for i in os.listdir(rootPath):
        _path = os.path.join(rootPath, i)
        if os.path.isdir(_path):
            match_ele = ele_pattern.match(i)
            if match_ele:
                ele_list.append([_path, match_ele.group(1)])
            else:
                ele_list.extend(SearchELE(_path, ele_pattern))

    return ele_list

In [116]:
def setup_logger(log_dir="./LOG", log_filename="file.log", file_level="WARNING", console_level="WARNING"):
    # 创建目录
    os.makedirs(log_dir, exist_ok=True)
    log_fd = os.path.join(log_dir, log_filename)

    logger.remove()
    # 如果已有日志文件，重命名添加时间戳
    if os.path.exists(log_fd):
        name, ext = os.path.splitext(log_filename)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        archived_name = f"{name}_{timestamp}{ext}"
        archived_path = os.path.join(log_dir, archived_name)
        os.rename(log_fd, archived_path)

    # 添加终端输出
    logger.add(sys.stdout, level=console_level, enqueue=True)

    # 添加文件输出
    logger.add(log_fd, level=file_level, encoding="utf-8", enqueue=True)

    return logger

## Run

In [ ]:
if True:
    setup_logger(log_dir="D:\Baihm\EISNN\LOG\DRTLoe_Process_Log")

# logger.remove()
# logger.add(sys.stdout, level="WARNING")
# logger.add("./LOG/file.log", rotation="10 MB", level="INFO")

2025-08-14 21:20:01.079 | WARNING  | __main__:<module>:38 - ELE[1/187]: 	02027452 - D:/Baihm/EISNN/Archive_New/2025\1月\02027452_归档
2025-08-14 21:20:01.081 | WARNING  | __main__:<module>:46 - FileAlreadyExistsWarning: 02027452 - 02027452_Outlier_Ver04_DRTLoe_Ver03.pt already exists.
2025-08-14 21:20:02.298 | WARNING  | __main__:<module>:38 - ELE[2/187]: 	02027453 - D:/Baihm/EISNN/Archive_New/2025\1月\02027453_归档
2025-08-14 21:20:02.300 | WARNING  | __main__:<module>:46 - FileAlreadyExistsWarning: 02027453 - 02027453_Outlier_Ver04_DRTLoe_Ver03.pt already exists.
2025-08-14 21:20:02.300 | WARNING  | __main__:<module>:24 - D:/Baihm/EISNN/Archive_New/2025\1月\06027742_归档\Outlier_Ver04\06027742_Outlier_Ver04.pt does not exist
2025-08-14 21:20:02.300 | WARNING  | __main__:<module>:24 - D:/Baihm/EISNN/Archive_New/2025\1月\09047284_归档\Outlier_Ver04\09047284_Outlier_Ver04.pt does not exist
2025-08-14 21:20:02.301 | WARNING  | __main__:<module>:24 - D:/Baihm/EISNN/Archive_New/2025\1月\09047285_归档\Out

In [118]:

# rootPath = "D:/Baihm/EISNN/Archive/"
# ele_list = SearchELE(rootPath)
# DATASET_SUFFIX = "Outlier_Ver04"

rootPath = "D:/Baihm/EISNN/Archive_New/"
ele_list = SearchELE(rootPath)
DATASET_SUFFIX = "Outlier_Ver04"

# rootPath = "D:/Baihm/EISNN/Invivo/"
# ele_list = SearchELE(rootPath, re.compile(r"(.+?)_Ver02"))
# DATASET_SUFFIX = "Outlier_Ver04"


n_ele = len(ele_list)
logger.info(f"Search in {rootPath} and find {n_ele:03d} electrodes")

In [119]:
# freq_list = np.linspace(1000,5000-1,101,dtype=int, endpoint=True)
# freq_list = np.linspace(0,5000-1,101,dtype=int, endpoint=True)
# freq_list = freq_list[1:]

DRT_SUFFIX = f"{DATASET_SUFFIX}_DRTLoe_Ver03"
SAVE_FLAG = True
FORCE_FLAG = False

In [120]:
RUN_FLAG = True
if RUN_FLAG:

    REALFLAG = True
    n_batch = 100
    n_point = 100


    pdf_x = np.logspace(-7, -2, 501)

    ##  In Vitro
    idx_base = 1000     # In vitro
    heat_range = [1,5.5]    # In vitro

    ##  In Vivo
    # idx_base = 0     # In vivo
    # heat_range = [1,6.5]    # in vivo
    
    for i in range(n_ele):
    # for i in range(0,1):
        # logger.info(f"ELE Begin: {ele_list[i][0]}")
        fd_pt = os.path.join(ele_list[i][0], DATASET_SUFFIX, f"{ele_list[i][1]}_{DATASET_SUFFIX}.pt")
        if not os.path.exists(fd_pt):
            logger.warning(f"{fd_pt} does not exist")
            continue
        
        data_pt = torch.load(fd_pt)
        _meta_group = data_pt["meta_group"]
        _data_group = data_pt["data_group"]


        ele_id  = _meta_group["ele_id"]
        elePath = _meta_group["elePath"]
        n_ch = _meta_group["n_ch"]      
        x_day_full = _meta_group["TimeSpan"]


        logger.warning(f"ELE[{i+1}/{n_ele}]: \t{ele_id} - {elePath}")


        # Storage path
        save_dir = f"{elePath}/{DRT_SUFFIX}/"
        pt_file_name = f"{ele_id}_{DRT_SUFFIX}.pt"
        os.makedirs(save_dir, exist_ok=True)
        if os.path.exists(os.path.join(save_dir, pt_file_name)):
            logger.warning(f"FileAlreadyExistsWarning: {ele_id} - {pt_file_name} already exists.")
            if SAVE_FLAG and not FORCE_FLAG:
                continue

        x_day_idx = np.array([(d - x_day_full[0]).days for d in x_day_full])


        available_channel = []
        for j in _data_group['Channels']:
            # if j!=44: continue
            try:
            # if 1:
                logger.warning(f"ELE[{ele_id}] - ch[{j:03d}] Begin") 
                channel_group_raw = _data_group[j]


                chData          =   channel_group_raw['chData']         
                eis_seq         =   channel_group_raw['eis_seq']        
                # eis_cluster     =   channel_group_raw['eis_cluster']    
                # eis_anomaly     =   channel_group_raw['eis_anomaly']    
                # leaf_anomaly    =   channel_group_raw['leaf_anomaly']   
                # seq_weird       =   channel_group_raw['seq_weird']      
                # seq_open        =   channel_group_raw['seq_open']       
                # seq_short       =   channel_group_raw['seq_short']      




                if chData.shape[2] != 5000:
                    logger.error(f"ELE[{ele_id}] - ch[{j}] with less than 5000 samples")
                    break
                 
                # Run DRT Analysis
                DRTdata_list = []
                EISdata_list = []
                IPDFdata_list = []
                PDFdata_list = []
                f = chData[0,0,:]
                for k in range(chData.shape[0]):
                    if k in eis_seq:
                        _Z = chData[k,1,:] + 1j*chData[k,2,:]
                        DRTdata, EISdata = DRT_Bootstrap(f, _Z, REALFLAG=REALFLAG, n_batch=n_batch, n_point=n_point, idx_base=idx_base)

                        tau_list    = [_drt[0,0:-1] for _drt in DRTdata]
                        R_list      = [_drt[1,0:-1] for _drt in DRTdata]
                        # tau_list    = [_drt[0,:] for _drt in DRTdata_o]
                        # R_list      = [_drt[1,:] for _drt in DRTdata_o]
                        tau_re_log  = DRT_Equidistribution(tau_list,
                                                        tau_quant=[0, 1], frac=0.1)

                        R_list_all     = np.concatenate(R_list, axis=0) / len(R_list)
                        tau_list_all   = np.concatenate(tau_list, axis=0)

                        _idx = np.argsort(tau_list_all)
                        tau_list_all = tau_list_all[_idx]
                        R_list_all = R_list_all[_idx]

                        ipdf_x, ipdf_p  = DRT_IPDF_Resample(tau_list_all, R_list_all, tau_re_log, frac= 0.05)
                        pdf_x, pdf_p    = DRT_IPDF2PDF(ipdf_x, ipdf_p, pdf_x)

                        ipdf_p = ipdf_p * len(R_list)
                        
                        
                        _idx = np.argsort(pdf_x)
                        pdf_x = pdf_x[_idx]
                        pdf_p = pdf_p[_idx]


                        DRTdata_list.append(DRTdata)
                        EISdata_list.append(EISdata)
                        IPDFdata_list.append(np.array([ipdf_x, ipdf_p]))
                        PDFdata_list.append(np.array([pdf_x, pdf_p]))
                    else:
                        DRTdata_list.append([])
                        EISdata_list.append([])
                        IPDFdata_list.append([])
                        PDFdata_list.append([])



                # Plot DRT Analysis Results
                fig = plt.figure(figsize=(16, 9), constrained_layout=True)
                fig.suptitle(f"ELE[{ele_id}] - ch[{j:03d}] LFDRT Analysis", fontsize=16, fontweight='bold')
                DRT_Plot_Batch(fig, DRTdata_list, EISdata_list, IPDFdata_list, PDFdata_list, x_day_idx, pdf_x, eis_seq, heat_range=heat_range)
                


                # Save Fig
                fig_name = f"DRT_{ele_id}_ch{j:03d}.png"
                
                os.makedirs(save_dir, exist_ok=True) 
                path = os.path.join(save_dir, fig_name)

                fig.savefig(path)
                plt.close('all') 

                # Data Saving
                channel_group_DRT = channel_group_raw
                channel_group_DRT['DRTlist']  = DRTdata_list
                channel_group_DRT['EISlist']  = EISdata_list
                channel_group_DRT['IPDFlist'] = IPDFdata_list
                channel_group_DRT['PDFlist']  = PDFdata_list

                available_channel.append(j)
                _data_group[j] = channel_group_DRT
                logger.info(f"ELE[{ele_id}] - ch[{j:03d}] Finished")
                
                # break
            except Exception as e:
                logger.error(f"ELE[{ele_id}] - ch[{j:03d}] Run with error: {e}")
                continue

        
        pt_store = {}
        _data_group['Channels'] = available_channel
        pt_store["meta_group"] = _meta_group
        pt_store["data_group"] = _data_group
        if SAVE_FLAG:
            torch.save(pt_store, os.path.join(save_dir, pt_file_name))

        del data_pt, _meta_group, _data_group, pt_store
        gc.collect()


C:\Users\Baihm\AppData\Local\Temp\4\ipykernel_6768\1387423846.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data_pt = torch.load(fd_pt)
C:\Users\Baihm\AppData\Local\T

KeyboardInterrupt: 

# Viewer

In [ ]:
VIEWER_FLAG = False
if VIEWER_FLAG:
    # ele_id = "02067447"
    # ch_id = 16    # Short with two phased
    # ch_id = 68    # Short Type II

    
    # ele_id = "06017758"
    # ch_id = 10    # Perfect

    
    ele_id = "01037160"
    ch_id = 0    # Short type I

    
    # ele_id = "11057712"
    # # ch_id = 26      # Normal to Short
    # ch_id = 107      # Normal to Short
    
    fd_pt = f"D:/Baihm/EISNN/Archive/{ele_id}_归档\\Outlier_Ver04_DRTLoe_Ver02\\{ele_id}_Outlier_Ver04_DRTLoe_Ver02.pt"

    # Read out
    data_pt = torch.load(fd_pt)
    _data_group = data_pt["data_group"]
    DRTdata_list = _data_group[ch_id]['DRTlist']
    EISdata_list = _data_group[ch_id]['EISlist']
    Loess_list = _data_group[ch_id]['Loesslist']
    
    # Plot
    fig = plt.figure(figsize=(16, 9), constrained_layout=True)
    text_axis = DRT_Plot_Batch(fig, DRTdata_list, EISdata_list, Loess_list, eis_seq)
    # DRT_Plot_Batch_3D(fig, DRTdata[:2], chData[:2,:,:])

